In [1]:
import pandas as pd

/Users/user/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
data_2020 = pd.read_excel('/Users/user/Desktop/community project/2020/2020.xlsx')
data_2021 = pd.read_csv('/Users/user/Desktop/community project/2021/2021.csv')

/Users/user/anaconda3/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
# Normalize text data to ensure better matching
data_2020['Palais des Congrès'] = data_2020['Palais des Congrès'].str.lower().str.strip()
data_2021['Culture'] = data_2021['Culture'].str.lower().str.strip()
data_2021['Variety'] = data_2021['Variety'].str.lower().str.strip()

# Find matches for 'Palais des Congrès' in both 'Culture' and 'Variety' columns of 2021
matches_culture = data_2020[data_2020['Palais des Congrès'].isin(data_2021['Culture'])]
matches_variety = data_2020[data_2020['Palais des Congrès'].isin(data_2021['Variety'])]

# Combine matches, removing duplicates if any row matches on both criteria
combined_matches = pd.concat([matches_culture, matches_variety]).drop_duplicates()
combined_matches


,Récoltes,Palais des Congrès,2020,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
82,2020-07-14 00:00:00,basilic,0.324,vertical,NaN,NaN,NaN,NaN,NaN
83,2020-07-15 00:00:00,basilic,0.818,vertical,NaN,NaN,NaN,NaN,NaN
84,2020-07-15 00:00:00,basilic,0.304,d,NaN,NaN,NaN,NaN,NaN
85,2020-07-29 00:00:00,basilic,2.25,vertical,NaN,NaN,NaN,NaN,NaN
86,2020-07-29 00:00:00,basilic,0.502,d,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
814,2020-09-28 00:00:00,tomate cerise,5.2,g,NaN,NaN,NaN,NaN,NaN
815,2020-09-28 00:00:00,tomate cerise,7.814,g,NaN,NaN,NaN,NaN,NaN
816,2020-10-05 00:00:00,tomate cerise,11.88,g,NaN,NaN,NaN,NaN,NaN
817,2020-10-14 00:00:00,tomate cerise,3.5,a,NaN,NaN,NaN,NaN,NaN


In [4]:
# First, normalize and prepare the 2020 data for aggregation by summing up quantities
# Assuming the column with numerical values ('nan' from previous outputs) refers to weights or quantities
data_2020['weight'] = pd.to_numeric(data_2020[combined_matches.columns[2]], errors='coerce')
data_2020_summed = data_2020.groupby('Palais des Congrès')['weight'].sum().reset_index()

# Now, prepare the 2021 data, checking if 'Poids (kg)' is already in the right format
data_2021_summed_culture = data_2021.groupby('Culture')['Poids (kg)'].sum().reset_index()
data_2021_summed_variety = data_2021.groupby('Variety')['Poids (kg)'].sum().reset_index()

# Merge the 2020 data with the 2021 data on matching 'Palais des Congrès' to 'Culture' and 'Variety'
matches_culture_sum = pd.merge(data_2020_summed, data_2021_summed_culture, left_on='Palais des Congrès', right_on='Culture', how='inner')
matches_variety_sum = pd.merge(data_2020_summed, data_2021_summed_variety, left_on='Palais des Congrès', right_on='Variety', how='inner')

# Combine both results and sum the weights if a category appears in both 'Culture' and 'Variety'
combined_sums = pd.concat([matches_culture_sum, matches_variety_sum])
final_sums = combined_sums.groupby('Palais des Congrès').agg({
    'weight': 'sum',  # Sum weights from 2020
    'Poids (kg)': 'sum'  # Sum weights from 2021
}).reset_index()

final_sums


,Palais des Congrès,weight,Poids (kg)
0,basilic,99.408,13.372
1,carotte,14.660,51.764
2,cerise de terre,57.988,21.000
3,chou-rave,145.242,116.769
4,concombre,3.136,62.521
5,courgette,7.968,136.762
6,fenouil,63.580,351.004
7,ficoide,30.308,17.712
8,gadelle,2.420,11.056
9,groseille,3.320,19.904
